In [1]:
import numpy as np
import os
from os import path
import cv2 as cv
import h5py
import pickle

In [2]:
datadir = '/home/user/data/female-4-casual/'

In [ ]:
# STEP 1: convert mp4 video to images

video_name = list(filter(lambda s: 'mp4' in s ,os.listdir(datadir)))[0]
video_path = path.join(datadir, video_name)
os.makedirs(path.join(datadir, 'images/00/'), exist_ok=True)
out_pattern = path.join(datadir, 'images/00/%06d.jpg')
cmd = f'/usr/bin/ffmpeg -i {video_path} -q:v 1 -start_number 0 {out_pattern}'
print(cmd)
os.system(cmd)


In [4]:
# STEP 2: dump masks from hdf5 file
os.makedirs(path.join(datadir, 'mask/00'), exist_ok=True)

import skimage.morphology as sm

f = h5py.File(path.join(datadir, 'masks.hdf5'), 'r')
d = f['masks']
for i in range(len(d)):
    out_path = path.join(datadir, f'mask/00/{i:06d}.png')
    mask = d[i][:,:,None]
    mask = sm.binary_erosion(mask)
    mask = mask.astype(np.uint8) * 255
    cv.imwrite(out_path, mask)

In [ ]:
# STEP 3: write smpl param to datadir

import sys
sys.path.append('..')
import scene.smpl as SMPL
smpl_path = [
    '../smpl_model/smpl/SMPL_FEMALE.pkl',
    '../smpl_model/smpl/SMPL_MALE.pkl',
]
SMPL.init(smpl_path[0] if 'female' in datadir else smpl_path[1])
print(smpl_path[0] if 'female' in datadir else smpl_path[1])

from scipy.spatial.transform import Rotation
os.makedirs(path.join(datadir, 'params'), exist_ok=True)

len_mask = len(f['masks'])

f = h5py.File(path.join(datadir, 'reconstructed_poses.hdf5'), 'r')
poses, trans, shapes = f['pose'][:], f['trans'][:], f['betas'][:]
print(len_mask, len(poses))

joints, _ = SMPL.get_joint_vert(SMPL.big_poses, shapes)

poses = poses[len(poses) - len_mask:]
trans = trans[len(trans) - len_mask:]
frame_ids = np.arange(0, len(poses))
for i in range(len(poses)):
    smpl = {}
    ##########
    smpl['Rh'] = np.copy(poses[i,:3])[None]
    smpl['R'] = Rotation.from_rotvec(smpl['Rh']).as_matrix()
    smpl['poses'] = np.copy(poses[i][None])
    smpl['poses'][0,:3] = 0
    smpl['shapes'] = shapes[None]

    root = joints[0]
    smpl['Th'] = trans[i] - (smpl['R'] @ root - root)
    smpl['Th'] = smpl['Th']
    ##########

    for key in smpl:
        if isinstance(smpl[key], np.ndarray):
            smpl[key] = smpl[key].astype(np.float32)   
            if i == 0:
                print(key, smpl[key].shape)

    np.save(path.join(datadir, f'params/{i}.npy'), smpl, allow_pickle=True)

In [ ]:
# STEP 4: annots.npy, including camera infos

with open(path.join(datadir, 'camera.pkl'), 'rb') as file:
    obj = pickle.load(file, encoding='latin1')

ndata = {}

T_list, R_list, K_list, D_list = [], [], [], []
for cam_id in range(1):
    T, R = obj['camera_t'], obj['camera_rt']
    D = obj['camera_k'].astype(np.float32)[None]
    focalx, focaly = obj['camera_f']
    W, H = obj['width'], obj['height']
    cx, cy = obj['camera_c']

    R = Rotation.from_rotvec(R).as_matrix()
    T = -R.T @ T
    R = R.T
    T = (T * 1000)[:,None]

    K = np.array([[focalx, 0, cx],
                  [0, focaly, cy],
                  [0,0,1]], dtype=np.float32)

    T_list.append(T)
    R_list.append(R)
    D_list.append(D)
    K_list.append(K)

cams_dict = dict(T=T_list, R=R_list, K=K_list, D=D_list)
ndata['cams'] = cams_dict

allfile_list = []
print(f'frames {len(frame_ids)}')
for frame_id in frame_ids:
    file_list = [] 
    for cam_id in range(1):
        file_list.append(f'images/{cam_id:02d}/{frame_id:06d}.jpg')
    allfile_list.append(dict(ims=file_list))
ndata['ims'] = allfile_list

np.save(path.join(datadir, 'annots.npy'), ndata, allow_pickle=True)